In [72]:
import os
import math

import pandas as pd
import numpy as np

## Работа с court cases samples 

In [36]:
df_list = []
for pkl in os.listdir('success'):
    if pkl == ".DS_Store":  # for MacOS-specific file
        continue
    df = pd.json_normalize(pd.read_pickle('success/'+pkl))
    df_list.append(df)

success/7736609147_b.pkl
success/5028024427_b.pkl
success/7622005092.pkl
success/7455005950_b.pkl
success/2111000271_b.pkl
success/3607001642.pkl
success/5030002212_b.pkl
success/7415064205_b.pkl
success/2536163593.pkl
success/5019015634.pkl
success/5414100369_b.pkl
success/2347009476_b.pkl
success/4321000070_b.pkl
success/1429006530_b.pkl
success/3505003270.pkl
success/7722651420.pkl
success/6732030100_b.pkl
success/1001227084.pkl
success/6907010474_b.pkl
success/1701050442_b.pkl
success/7701651532_b.pkl
success/4029028408.pkl
success/7326037640_b.pkl
success/4521003135.pkl
success/5535007195.pkl
success/3525203863.pkl
success/7724793607_b.pkl
success/5044075933.pkl
success/7701223216.pkl
success/3507005160_b.pkl
success/6105007117.pkl
success/1203000142_b.pkl
success/2107004611_b.pkl
success/6452018325.pkl
success/.DS_Store
success/1411004211_b.pkl
success/7714149601_b.pkl
success/6730010179.pkl
success/3721002810.pkl
success/6658410536.pkl
success/7453079153.pkl
success/7621009707_b

In [37]:
df_inn = pd.concat(df_list, axis=0, ignore_index=True)
df_inn

,inn,cases_list,error,n_requests
0,7736609147,"[{'caseNo': 'А40-105126/2021', 'resultType': '...",None,1
1,5028024427,"[{'caseNo': 'А41-30862/2019', 'resultType': 'И...",None,1
2,7622005092,[],None,1
3,7455005950,"[{'caseNo': 'А76-23313/2021', 'resultType': 'Н...",None,1
4,2111000271,"[{'caseNo': 'А79-5049/2016', 'resultType': 'Вы...",None,1
...,...,...,...,...
188,5406339932,"[{'caseNo': 'А45-28666/2017', 'resultType': 'Н...",None,1
189,7701038326,"[{'caseNo': 'А40-64050/2011', 'resultType': 'П...",None,1
190,5045021088,"[{'caseNo': 'А41-68887/2014', 'resultType': 'П...",None,1
191,5009033730,"[{'caseNo': 'А41-60703/2019', 'resultType': 'Н...",None,1


In [1]:
# отберем те признаки, к-ые могут быть полезны:
# 
# resultType(не удалось определить, выиграно, проиграно) -- посчитать кол-во каждого варианта:
# resultType_проиграно, resultType_не_удалось_опеределить, resultType_выиграно
# isActive -- по этому признаку можно посчитать кол-во активных разбирательств
# documents_num -- много документов, можно посчитать их кол-во
# Сколько раз соответствующий ИНН был:
# case_sides.type.0 -- истцом
# case_sides.type.1 -- ответчиком
# case_sides.type.2 -- свидетелем
# также посчитаем sum для каждого resultType


In [219]:
df_inn["cases_list"][6][1]

{'caseNo': 'А40-105239/2021',
 'resultType': 'Проиграно',
 'caseDate': '2021-05-20T00:00:00',
 'caseType': {'code': '6',
  'name': 'экономические споры по гражданским правоотношениям'},
 'sum': 944631.65,
 'currency': 643,
 'isActive': False,
 'currentInstance': 'АС города Москвы',
 'instanceDate': '2021-06-28T00:00:00',
 'documents': [{'document_name': 'Решение суда первой инстанции',
   'document_date': '2021-06-28T00:00:00',
   'document': 'http://kad.arbitr.ru/kad/pdfdocument/6a0b3ca2-52e8-4ff0-9e26-8a4e3f991f63/A40-105239-2021_20210628_Reshenija_i_postanovlenija.pdf',
   'judge_name': 'Дубовик О. В.',
   'court_name': 'АС города Москвы'},
  {'document_name': 'Резолютивная часть решения суда первой инстанции',
   'document_date': '2021-06-24T00:00:00',
   'document': 'http://kad.arbitr.ru/kad/pdfdocument/80f4fd16-1381-491d-b320-faa67e3fb6c0/A40-105239-2021_20210624_Prochie_sudebnye_dokumenty.pdf',
   'judge_name': 'Дубовик О. В.',
   'court_name': 'АС города Москвы'},
  {'document_

In [137]:
cases_list = df_inn["cases_list"].values.tolist()

In [220]:
# Собираем нужные признаки
documents_len_list = []
result_type_proigrano_list = []
num_of_active_cases_list = []
result_type_viygrano_list = []
result_type_ne_udalos_opredelit_list = []
case_side_type_0_list = []
case_side_type_1_list = []
case_side_type_2_list = []
proigrano_sum_list = []
viygrano_sum_list = []

for case in cases_list:
    len_of_case = len(case)
    
    documents_len = 0
    result_type_proigrano = 0
    result_type_viygrano = 0
    active_cases = 0
    result_type_ne_udalos_opredelit = 0
    case_side_type_0 = 0
    case_side_type_1 = 0
    case_side_type_2 = 0
    proigrano_sum = 0
    viygrano_sum = 0
    
    for case_ind in range(len_of_case):
        d = case[case_ind]
        
        if d["documents"]:
            documents_len += len(d["documents"])

        if d["resultType"] == 'Проиграно':
            result_type_proigrano += 1
            proigrano_sum += d["sum"]
            
        if d["resultType"] == "Выиграно":
            result_type_viygrano += 1
            viygrano_sum += d["sum"]
            
        if d["resultType"] == "Не удалось определить":
            result_type_ne_udalos_opredelit += 1
            
        if int(d['isActive']) == 1:
            active_cases += 1
        
        if len(d["case_sides"]) > 1:
            case_side_type = d["case_sides"][1]['type']

        if len(d["case_sides"]) == 1:
            case_side_type = d["case_sides"][0]['type']

        if len(d["case_sides"]) == 0:
            case_side_type = -1

            
        if case_side_type == 0:
            case_side_type_0 += 1
        if case_side_type == 1:
            case_side_type_1 += 1
        if case_side_type == 2:
            case_side_type_2 += 1
        
    documents_len_list.append(documents_len)
    result_type_proigrano_list.append(result_type_proigrano)
    result_type_viygrano_list.append(result_type_viygrano)
    result_type_ne_udalos_opredelit_list.append(result_type_ne_udalos_opredelit)
    num_of_active_cases_list.append(active_cases)
    
    case_side_type_0_list.append(case_side_type_0)
    case_side_type_1_list.append(case_side_type_1)
    case_side_type_2_list.append(case_side_type_2)
    
    proigrano_sum_list.append(proigrano_sum)
    viygrano_sum_list.append(viygrano_sum)
        

In [226]:
proigrano_sum_list = [round(x, 2) for x in proigrano_sum_list]
viygrano_sum_list = [round(x, 2) for x in viygrano_sum_list]

In [230]:
df_inn["documents_len"] = documents_len_list
df_inn["result_type_proigrano"] = result_type_proigrano_list
df_inn["result_type_viygrano"] = result_type_viygrano_list
df_inn["result_type_ne_udalos_opredelit"] = result_type_ne_udalos_opredelit_list
df_inn["num_of_active_cases"] = num_of_active_cases_list
df_inn["case_side_type_0"] = case_side_type_0_list
df_inn["case_side_type_1"] = case_side_type_1_list
df_inn["case_side_type_2"] = case_side_type_2_list
df_inn["proigrano_sum"] = proigrano_sum_list
df_inn["viygrano_sum"] = viygrano_sum_list

In [232]:
cases_len_list = [len(x) for x in df_inn["cases_list"]]

In [233]:
df_inn["cases_len"] = cases_len_list

In [235]:
# бесполезный признак, так как одни None 
df_inn["error"].value_counts()

Series([], Name: error, dtype: int64)

In [236]:
# удалим уже ненужные признаки
df_inn = df_inn.drop(["error", "cases_list"], axis=1)

In [237]:
df_inn.head()

,inn,n_requests,documents_len,result_type_proigrano,result_type_viygrano,result_type_ne_udalos_opredelit,num_of_active_cases,case_side_type_0,case_side_type_1,case_side_type_2,proigrano_sum,viygrano_sum,cases_len
0,7736609147,1,253,23,6,23,1,2,57,0,51153662.45,10395854.55,59
1,5028024427,1,99,1,0,1,1,1,1,0,4269304.88,0.00,4
2,7622005092,1,0,0,0,0,0,0,0,0,0.00,0.00,0
3,7455005950,1,76,1,2,2,1,1,5,0,7779.05,808000.00,6
4,2111000271,1,150,1,2,9,0,3,10,0,2439854.48,2439854.48,13


## Работа с account samples dataframe 

In [248]:
df_accounts_samples = pd.read_csv("df_accounts_sample.csv")
df_accounts_samples = df_accounts_samples.drop(["Unnamed: 0"], axis=1)
df_accounts_samples

,inn,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei
0,2465002653,2012,0.0,113756,122240,384
1,3505003270,2012,21461.0,20211,81132,384
2,3517003250,2012,0.0,0,48063,384
3,5501078737,2012,0.0,288763,419522,384
4,6140019517,2012,0.0,0,11089,384
...,...,...,...,...,...,...
1030,7842353970,2020,0.0,2548,16411,384
1031,8908001998,2019,NaN,8636,130731,384
1032,8908001998,2020,NaN,14318,128891,384
1033,9102060446,2019,6000.0,229769,308868,384


In [114]:
agg_fn = {'inn': 'first', 'year': 'first', 'long_term_liabilities_fiscal_year': 'first', 
          'short_term_liabilities_fiscal_year': 'first', 'balance_assets_fiscal_year': 'first', 'okei': 'first'}

dfss = df_accounts_samples.groupby(df_accounts_samples["inn"]).aggregate(agg_fn)

In [115]:
dfss

,inn,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei
inn,,,,,,
1001003743,1001003743,2013,0.0,1080,4399,384
1020010136,1020010136,2017,0.0,36750,14769,384
1203000142,1203000142,2017,252652.0,165670,493782,384
1302000640,1302000640,2012,0.0,0,32400,384
1411004211,1411004211,2012,0.0,0,167857,384
...,...,...,...,...,...,...
8617004112,8617004112,2017,0.0,4853,6230,384
8901998318,8901998318,2017,0.0,259,259,384
8908001998,8908001998,2012,0.0,7668,17507,384


## Работа с bankruptcies dataframe

In [240]:
df_bankruptcies = pd.read_csv("df_bankruptcies_sample.csv")

In [241]:
df_bankruptcies = df_bankruptcies.drop(["Unnamed: 0"], axis=1)
df_bankruptcies = df_bankruptcies.drop(["bankrupt_id"], axis=1)
df_bankruptcies

,bancrupt_year,inn
0,2015,5620006322
1,2015,2111008200
2,2016,5414100369
3,2019,2347009476
4,2018,7224008489
...,...,...
101,2021,7728812071
102,2019,6154086295
103,2020,6627020656
104,2021,2465002653


## Объединяем все

In [242]:
df = df_inn.join(df_bankruptcies.set_index('inn'), on='inn')
df

,inn,n_requests,documents_len,result_type_proigrano,result_type_viygrano,result_type_ne_udalos_opredelit,num_of_active_cases,case_side_type_0,case_side_type_1,case_side_type_2,proigrano_sum,viygrano_sum,cases_len,bancrupt_year
0,7736609147,1,253,23,6,23,1,2,57,0,51153662.45,10395854.55,59,2021.0
1,5028024427,1,99,1,0,1,1,1,1,0,4269304.88,0.00,4,2019.0
2,7622005092,1,0,0,0,0,0,0,0,0,0.00,0.00,0,NaN
3,7455005950,1,76,1,2,2,1,1,5,0,7779.05,808000.00,6,2016.0
4,2111000271,1,150,1,2,9,0,3,10,0,2439854.48,2439854.48,13,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,5406339932,1,375,25,0,37,0,6,63,0,34568136.72,0.00,69,2015.0
189,7701038326,1,3,1,0,0,0,0,1,0,50000.00,0.00,1,NaN
190,5045021088,1,41,2,1,3,0,0,6,0,67447.33,0.00,6,NaN
191,5009033730,1,4870,4,21,32,4,9,49,5,794277.24,64052835.29,86,NaN


In [243]:
predicted_list = [1 if not math.isnan(x) else 0 for x in df["bancrupt_year"].values.tolist()]
df["is_bankrupt"] = predicted_list

In [244]:
df

,inn,n_requests,documents_len,result_type_proigrano,result_type_viygrano,result_type_ne_udalos_opredelit,num_of_active_cases,case_side_type_0,case_side_type_1,case_side_type_2,proigrano_sum,viygrano_sum,cases_len,bancrupt_year,is_bankrupt
0,7736609147,1,253,23,6,23,1,2,57,0,51153662.45,10395854.55,59,2021.0,1
1,5028024427,1,99,1,0,1,1,1,1,0,4269304.88,0.00,4,2019.0,1
2,7622005092,1,0,0,0,0,0,0,0,0,0.00,0.00,0,NaN,0
3,7455005950,1,76,1,2,2,1,1,5,0,7779.05,808000.00,6,2016.0,1
4,2111000271,1,150,1,2,9,0,3,10,0,2439854.48,2439854.48,13,2014.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,5406339932,1,375,25,0,37,0,6,63,0,34568136.72,0.00,69,2015.0,1
189,7701038326,1,3,1,0,0,0,0,1,0,50000.00,0.00,1,NaN,0
190,5045021088,1,41,2,1,3,0,0,6,0,67447.33,0.00,6,NaN,0
191,5009033730,1,4870,4,21,32,4,9,49,5,794277.24,64052835.29,86,NaN,0


In [245]:
# Очевидно, что нужно удалить признак bancrupt_year, тк любой классификатор сразу же найдет разделяющую 
# поверхность просто из-за одного фактора (если года нет, то компания банкрот). Чтобы было честно:

In [246]:
df = df.drop(["bancrupt_year"], axis=1)

In [247]:
df

,inn,n_requests,documents_len,result_type_proigrano,result_type_viygrano,result_type_ne_udalos_opredelit,num_of_active_cases,case_side_type_0,case_side_type_1,case_side_type_2,proigrano_sum,viygrano_sum,cases_len,is_bankrupt
0,7736609147,1,253,23,6,23,1,2,57,0,51153662.45,10395854.55,59,1
1,5028024427,1,99,1,0,1,1,1,1,0,4269304.88,0.00,4,1
2,7622005092,1,0,0,0,0,0,0,0,0,0.00,0.00,0,0
3,7455005950,1,76,1,2,2,1,1,5,0,7779.05,808000.00,6,1
4,2111000271,1,150,1,2,9,0,3,10,0,2439854.48,2439854.48,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,5406339932,1,375,25,0,37,0,6,63,0,34568136.72,0.00,69,1
189,7701038326,1,3,1,0,0,0,0,1,0,50000.00,0.00,1,0
190,5045021088,1,41,2,1,3,0,0,6,0,67447.33,0.00,6,0
191,5009033730,1,4870,4,21,32,4,9,49,5,794277.24,64052835.29,86,0


In [116]:
dfss

,inn,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei
inn,,,,,,
1001003743,1001003743,2013,0.0,1080,4399,384
1020010136,1020010136,2017,0.0,36750,14769,384
1203000142,1203000142,2017,252652.0,165670,493782,384
1302000640,1302000640,2012,0.0,0,32400,384
1411004211,1411004211,2012,0.0,0,167857,384
...,...,...,...,...,...,...
8617004112,8617004112,2017,0.0,4853,6230,384
8901998318,8901998318,2017,0.0,259,259,384
8908001998,8908001998,2012,0.0,7668,17507,384


In [250]:
df2 = df.join(dfss.set_index('inn'), on='inn')

In [251]:
df2

,inn,n_requests,documents_len,result_type_proigrano,result_type_viygrano,result_type_ne_udalos_opredelit,num_of_active_cases,case_side_type_0,case_side_type_1,case_side_type_2,proigrano_sum,viygrano_sum,cases_len,is_bankrupt,year,long_term_liabilities_fiscal_year,short_term_liabilities_fiscal_year,balance_assets_fiscal_year,okei
0,7736609147,1,253,23,6,23,1,2,57,0,51153662.45,10395854.55,59,1,2012.0,0.0,75255.0,85541.0,384.0
1,5028024427,1,99,1,0,1,1,1,1,0,4269304.88,0.00,4,1,2013.0,0.0,59825.0,48360.0,384.0
2,7622005092,1,0,0,0,0,0,0,0,0,0.00,0.00,0,0,NaN,NaN,NaN,NaN,NaN
3,7455005950,1,76,1,2,2,1,1,5,0,7779.05,808000.00,6,1,2013.0,38411.0,4198.0,45890.0,384.0
4,2111000271,1,150,1,2,9,0,3,10,0,2439854.48,2439854.48,13,1,2012.0,0.0,0.0,6122.0,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,5406339932,1,375,25,0,37,0,6,63,0,34568136.72,0.00,69,1,2012.0,2791.0,289333.0,287226.0,384.0
189,7701038326,1,3,1,0,0,0,0,1,0,50000.00,0.00,1,0,2012.0,0.0,0.0,12528.0,384.0
190,5045021088,1,41,2,1,3,0,0,6,0,67447.33,0.00,6,0,2012.0,0.0,0.0,1597.0,384.0
191,5009033730,1,4870,4,21,32,4,9,49,5,794277.24,64052835.29,86,0,2012.0,1992651.0,830163.0,3884450.0,384.0


In [252]:
df2.to_csv("df_after_collecting_data.csv")